-----
This Notebook provides some sample code you can use (and extend) to check your maps
-----
-----

In [1]:
!pip install geojson
import geojson
import pandas as pd
import numpy as np

### Checking Precinct Assignment for my map

In [2]:
#assignment will contain the precinct-district assignments from my map
assignment = pd.read_csv('badlypaintedmapexample.csv')
#assignment = pd.read_csv('incompletemap.csv')
assignment.head(10)

,GEOID20,District
0,34021030004,20
1,34021030007,1
2,34021030003,1
3,34019130002,1
4,34019130001,1
5,34019035003,2
6,34019085002,1
7,34019035001,1
8,34019085003,1
9,34019035005,1


In [3]:
print(f"- We have {assignment.shape[0]} number of precincts assigned to {assignment.District.nunique()} districts.")
#print(f"- There are {np.count_nonzero(assignment.isnull().values.ravel())} missing values in the dataset.\n")
print(f"- All the precint ids in the dataset are unique is {assignment.shape[0] == assignment.GEOID20.nunique()}")
print(f"- {6754-assignment.shape[0]} Districts are unassigned (including 415 missing/corrupted precincts)")

- We have 6343 number of precincts assigned to 40 districts.
- All the precint ids in the dataset are unique is True
- 411 Districts are unassigned (including 415 missing/corrupted precincts)


In [4]:
#Number of Precincts per district
assignment.groupby(by="District").size()

District
1      91
2      46
3      70
4      42
5     238
6     647
7     158
8     357
9      78
10    110
11     99
12     92
13    467
14    251
15    242
16      5
17    790
18    300
19     62
20    129
21    184
22     58
23    375
24    304
25     56
26    133
27     53
28     50
29     77
30     92
31     42
32     96
33     26
34     60
35     42
36    104
37     16
38    147
39     83
40     71
dtype: int64

### Analyzing Precinct Data per district

In [5]:
#pre_data will contain the precinct demographics and historical election data.

###final_NJ_2020_State_Legislative_data.csv has the 2020 data
pre_data = pd.read_csv('final_NJ_2020_State_Legislative_data.csv')

pre_data.head(5)

,GEOID20,District,FriendlyId,Total_2016-2021_Comp,Dem_2016-2021_Comp,Rep_2016-2021_Comp,Total_2021_Gov,Dem_2021_Gov,Rep_2021_Gov,Total_2020_Sen,...,VTDI20,NAME20,NAMELSAD20,LSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20
0,34033020001,3,34033020001,523.0,103.0,409.0,484.0,93.0,385.0,589.0,...,P,Lower Alloways Creek township voting district 1,Lower Alloways Creek township voting district 1,0.0,G5240,S,67138130.0,45124792.0,39.488403,-75.477086
1,34033001501,3,34033001501,570.0,172.0,386.0,473.0,136.0,334.0,664.0,...,A,Elsinboro township voting district 1,Elsinboro township voting district 1,0.0,G5240,N,30901777.0,3656526.0,39.533317,-75.498478
2,34033042009,3,34033042009,669.0,211.0,443.0,440.0,106.0,329.0,867.0,...,A,Pennsville township voting district 9,Pennsville township voting district 9,0.0,G5240,N,28489413.0,7652453.0,39.597819,-75.511333
3,34033000703,3,34033000703,456.0,220.0,228.0,254.0,106.0,148.0,631.0,...,A,Carneys Point township voting district 3,Carneys Point township voting district 3,0.0,G5240,N,5014278.0,267398.0,39.687697,-75.473822
4,34033042001,3,34033042001,714.0,286.0,413.0,993.0,467.0,513.0,647.0,...,A,Pennsville township voting district 1,Pennsville township voting district 1,0.0,G5240,N,6777631.0,751223.0,39.671387,-75.494625


In [6]:
pre_data['']

KeyError: ''

### Calculating the population of each district of my map

In [ ]:
##This code shows you an example of how to calculate the population of a district, you can adapt it to count the partisan support.

population=0
sum_pop=0
for districts in (assignment.groupby(by="District")):
    #print(districts)
    for precinct in districts[1].iterrows():
        #print(precinct[1][0])
        if(pre_data.loc[pre_data.GEOID20==str(precinct[1][0]),'Total_2020_TotalAdj'].any()):
            population += pre_data.loc[pre_data.GEOID20==str(precinct[1][0]),'Total_2020_TotalAdj'].values[0]
            #print(type(population))
    print("The population of District "+str(districts[0])+ " is "+str(population))
    sum_pop+=population
    population=0
print("Total population is "+str(sum_pop))

------
Checking the geography of the map
------
------

### Are two precincts contiguous?

In [ ]:
!pip install shapely
!pip install PyShp

In [ ]:
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
#import StringIO


shpfile = 'nj_vtd_2020_bound.shp'
dbffile = 'nj_vtd_2020_bound.dbf'
shxfile = 'nj_vtd_2020_bound.shx'
shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
print(shpfile)

In [ ]:
field_names = []
for f in shpfile.fields[1:]:
    field_names.append((f[0]))
print(field_names)

precinct_boundaries={}
count=0
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    precinct_boundaries[rec[3]]=geom
    count=count+1
    
##print the coordinated of a precinct polygon
print(Polygon(shape(precinct_boundaries['34003060003'])))

In [ ]:
###Examples
a = Polygon(shape(precinct_boundaries['34003060003']))
b = Polygon(shape(precinct_boundaries['34007043046']))
c = Polygon(shape(precinct_boundaries['34007043047']))

#print(shape(precinct_boundaries['34029102002']))
                  
#b = Polygon(precint_boundaries['34007043046'])
print(a.touches(b))
print(c.touches(b))

#### Function to check if two precinct overlap (needs precinct_boundaries to be instanciated)

In [ ]:
def is_contiguous_precinct(p1,p2,precinct_boundaries):
    #print(shape(precinct_boundaries[p1]).type)
    try:
        if shape(precinct_boundaries[p1]).type == 'Polygon':
            a = Polygon(shape(precinct_boundaries[p1]))
        else:
            return False
        if shape(precinct_boundaries[p2]).type == 'Polygon':
            b = Polygon(shape(precinct_boundaries[p2]))
        else:
            return False
        return(a.touches(b))
    except KeyError: return False

In [ ]:
print(is_contiguous_precinct('34003060003','34007043046',precinct_boundaries))
print(is_contiguous_precinct('34007043047','34007043046',precinct_boundaries))
print(is_contiguous_precinct('34003060003','34003060002',precinct_boundaries))
print(is_contiguous_precinct('34007043047','34003060003',precinct_boundaries))
print(is_contiguous_precinct('34041080003','34005020001',precinct_boundaries))
print(is_contiguous_precinct('34041080003','34005075101',precinct_boundaries))



#### Function to find contiguous districts (needs precinct_boundaries to be instanciated)

In [ ]:
## This code will tell you how many precinct are contiguous to a given precinct
def contiguous_precincts(p1,precinct_boundaries):
    count=0
    neighbors=[]
    #district;
    for p in range(0,len(pre_data)):
        #print(pre_data.iloc[p].GEOID20)
        if(is_contiguous_precinct(p1,pre_data.iloc[p].GEOID20,precinct_boundaries)):
            count+=1
            neighbors.append(pre_data.iloc[p].GEOID20)
    return([count,neighbors])


In [ ]:
print(contiguous_precincts('34005070001',precinct_boundaries))
print(contiguous_precincts('34041080003',precinct_boundaries))

In [7]:
import pickle
picklename = 'FinalPrecinctContiguous.p'
contiguous_precincts= pickle.load(open(picklename,"rb"))

In [17]:
contiguous_precincts
type(contiguous_precincts)


6754

In [ ]:
#here we can merge/recombine and cut population in half (first attempt)
#finding what districts are contiguous
import csv

conti_p = {}
for precincts in (assignment.groupby(by="GEOID20")):
    #for precinct in districts[1].iterrows():

    p1 = precincts[0]
    #print(p1)
    conti_p[p1] = contiguous_precincts(p1, precinct_boundaries)[1]
#field_name = ['Precinct', 'Count', 'Neighbors']

#with open('conti_p.csv', 'w') as csvfile:
#    writer = csv.DictWriter(csvfile, fieldnames = field_names)
#    writer.writeheader()
#    for data in conti_p:
#            writer.writerow(data)

In [ ]:
print(list(conti_p.values()))

In [ ]:
with open('conti_p.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=conti_p.keys())
    writer.writeheader()
    writer.writerow(conti_p)

In [ ]:
def contiguous_districts(p1,precinct_boundaries):
    count=0
    neighbors=[]
    for p in range(0,len(pre_data)):
        district = 0;
        #print(pre_data.iloc[p].GEOID20)
        if(is_contiguous_precinct(p1,pre_data.iloc[p].GEOID20,precinct_boundaries)):
            count+=1
            neighbors.append(pre_data.iloc[p].GEOID20)
            district = pre_data.iloc[p].District
    return([count,neighbors, district])

In [ ]:
print(contiguous_districts('34005070001',precinct_boundaries))
print(contiguous_districts('34041080003',precinct_boundaries))

In [ ]:
#gerrymandering method


In [ ]:
#output has to be GEOID20 and district 